# Loading Libraries 

In [ ]:
import tensorflow
import keras
import numpy as np
import warnings
import os
import filecmp
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.preprocessing import LabelBinarizer


warnings.filterwarnings("ignore")

# Loading Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls '/content/drive/MyDrive/Mask_Dataset'

Face_Mask  No_Mask


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
categories = ["Face_Mask", "No_Mask"]
Z

In [ ]:
directory = "/content/drive/MyDrive/Mask_Dataset"
categories = ["Face_Mask", "No_Mask"]

epochs_value=20
BS=32
data = []
tar = []

for category in categories:
    path = os.path.join(directory, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224, 3), color_mode='rgb')
        image = img_to_array(image)
        image = preprocess_input(image)    
        data.append(image)
        tar.append(category)

lb = LabelBinarizer() 
tar = lb.fit_transform(tar)
tar = to_categorical(tar)

data = np.array(data, dtype="float32")
tar = np.array(tar)

# Training The Model

In [ ]:
from sklearn.model_selection import train_test_split
(train_data,test_data,train_target,test_target)=train_test_split(data,tar,test_size=0.2, stratify=tar, random_state=42)


from keras_preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Flatten,Dropout,Conv2D,MaxPooling2D
from tensorflow.keras.optimizers import SGD

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(2, activation='sigmoid'))

opt = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])


H=model.fit(
    aug.flow(train_data,train_target,batch_size=BS),
    steps_per_epoch=len(train_data)//BS,
    validation_data=(test_data,test_target),
    validation_steps=len(test_data)//BS,
    epochs=epochs_value)

Epoch 1/20
48/48 [==============================] - 77s 2s/step - loss: 0.3345 - accuracy: 0.8384 - val_loss: 0.1472 - val_accuracy: 0.9436
Epoch 2/20
48/48 [==============================] - 76s 2s/step - loss: 0.1340 - accuracy: 0.9594 - val_loss: 0.0125 - val_accuracy: 0.9949
Epoch 3/20
48/48 [==============================] - 76s 2s/step - loss: 0.1046 - accuracy: 0.9666 - val_loss: 0.0219 - val_accuracy: 0.9923
Epoch 4/20
48/48 [==============================] - 76s 2s/step - loss: 0.0747 - accuracy: 0.9764 - val_loss: 0.0069 - val_accuracy: 0.9974
Epoch 5/20
48/48 [==============================] - 76s 2s/step - loss: 0.0791 - accuracy: 0.9758 - val_loss: 0.0175 - val_accuracy: 0.9949
Epoch 6/20
48/48 [==============================] - 76s 2s/step - loss: 0.0644 - accuracy: 0.9810 - val_loss: 0.0056 - val_accuracy: 1.0000
Epoch 7/20
48/48 [==============================] - 79s 2s/step - loss: 0.0485 - accuracy: 0.9876 - val_loss: 0.0071 - val_accuracy: 0.9974
Epoch 8/20
48/48 [==

In [ ]:
prediction = model.predict(test_data, batch_size=BS)
prediction = np.argmax(prediction, axis=1)



from sklearn.metrics import classification_report
print(classification_report(test_target.argmax(axis=1), prediction,
    target_names=lb.classes_))


model.save("TISB_HACKS.model", save_format="h5")

              precision    recall  f1-score   support

   Face_Mask       1.00      1.00      1.00       190
     No_Mask       1.00      1.00      1.00       200

    accuracy                           1.00       390
   macro avg       1.00      1.00      1.00       390
weighted avg       1.00      1.00      1.00       390



In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  
 

In [1]:
import imutils
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import cv2
import os
import namegenerator
import time
from datetime import datetime

parent_directory = r"D:\Nachiket\Coding\TISB_Hacks\Extracted" 


            
def detect_and_predict_mask(frame, faceNet, maskNet):

    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
        (104.0, 177.0, 123.0))

    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)


    faces = []
    locs = []
    preds = []

    for i in range(0, detections.shape[2]):

        confidence = detections[0, 0, i, 2]


        if confidence > 0.5:

            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")


            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))


            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)


            faces.append(face)
            locs.append((startX, startY, endX, endY))

    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)
        


    return (locs, preds)

prototxtPath = r"deploy.prototxt"
weightsPath = r"res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

maskNet = load_model("TISB_HACKS.model")

print("start")

from IPython.display import Image
try:
  vs = take_photo()
  
  display(Image(filename))
except Exception as err:

  print(str(err))


while True:
        
    frame = vs.read()
    frame = imutils.resize(frame, width=400)


    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)


    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred
        
            
        label = "Mask" if mask > withoutMask else "No Mask"
            
            
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        

        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
    
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
        
    if key == ord("q"):
        break
    time.sleep(1)
    
cv2.destroyAllWindows()
vs.stop()

ModuleNotFoundError: ignored